In [73]:
from numba import njit, int32, prange
import numpy as np

def sum(a):
  s = 0
  for i in range(len(a)):
    s += a[i]
  return s

@njit(int32(int32[:])) # argument is an array of integers "int32[:]" and the output is an integer "int32(arguments)"
def sum_optimized(a):
  s = 0
  for i in range(len(a)):
    s += a[i]
  return s

@njit(int32(int32[:]), parallel=True)
def sum_parallel(a):
  s = 0
  for i in prange(len(a)):
    s += a[i]
  return s

a = np.arange(60_000)

print(sum_optimized(a), sum_parallel(a)) # verify that sum_parallel works (only one thread at a time can write to s)

%timeit sum(a)
%timeit sum_optimized(a)
%timeit sum_parallel(a)

1799970000 1799970000
9.04 ms ± 253 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
37.8 µs ± 625 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
28.9 µs ± 1.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [110]:
from numba.types import float64, int32
from numba import int32, void
float64, int32, void, None

(float64, int32, none, None)

In [38]:
import math

@njit(int32(int32[:], int32), parallel=True)
def sum_parallel_chunk(a, chunk_size):
  num_chunks = math.ceil(len(a)/chunk_size)
  for chunk_i in prange(num_chunks): # assign thread to one chunk
    s = 0 # each thread has its own s 
    for element_i in range(chunk_size): # position relative to chunk
      i = chunk_i * chunk_size + element_i # "absolute" position
      if i < len(a): # last chunk could be smaller than chunk_size
        s += a[i]
    print(f'sum of chunk {chunk_i} : {s}')
  return num_chunks

sum_parallel_chunk(a, chunk_size=100)

sum of chunk 0 : 4950sum of chunk 9 : 94950
sum of chunk 8 : 84950
sum of chunk 6 : 64950

sum of chunk 2 : 24950
sum of chunk 3 : 34950
sum of chunk 7 : 74950
sum of chunk 1 : 14950
sum of chunk 4 : 44950
sum of chunk 5 : 54950


10

In [49]:
import math

@njit(int32(int32[:], int32), parallel=True)
def sum_parallel_chunk(a, num_chunks):
  chunk_size = math.ceil(len(a)/num_chunks)
  print(f'chunk size: {chunk_size}')
  for chunk_i in prange(num_chunks): # assign thread to one chunk
    s = 0 # each thread has its own s 
    for element_i in range(chunk_size): # position relative to chunk
      i = chunk_i * chunk_size + element_i # "absolute" position
      if i < len(a): # last chunk could be smaller than chunk_size
        s += a[i]
    print(f'sum of chunk {chunk_i} : {s}')
  return num_chunks

sum_parallel_chunk(a, num_chunks=8)

chunk size: 125
sum of chunk 0 : 7750
sum of chunk 1 : 23375
sum of chunk 4 : 70250
sum of chunk 2 : 39000
sum of chunk 5 : 85875
sum of chunk 7 : 117125
sum of chunk 6 : 101500
sum of chunk 3 : 54625


8

In [76]:
@njit(int32(int32[:], int32), parallel=True)
def sum_parallel_chunk(a, num_chunks):
  chunk_size = math.ceil(len(a)/num_chunks)
  shared_sums = np.zeros(num_chunks)
  for chunk_i in prange(num_chunks): # assign thread to one chunk
    for element_i in range(chunk_size): # position relative to chunk
      i = chunk_i * chunk_size + element_i # "absolute" position
      if i < len(a): # last chunk could be smaller than chunk_size
        shared_sums[chunk_i] += a[i]
  # combine the results
  result = 0
  for s in shared_sums:
    result += s
  return result
  
a = np.arange(1000)
sum_parallel_chunk(a, num_chunks=10)

499500

In [81]:
a = np.ones(1_000_000, dtype=np.int32)
%timeit sum_optimized(a)
%timeit sum_parallel(a)
print('chunks')
%timeit sum_parallel_chunk(a, num_chunks=10)
%timeit sum_parallel_chunk(a, num_chunks=100)
%timeit sum_parallel_chunk(a, num_chunks=1000)
%timeit sum_parallel_chunk(a, num_chunks=10_000)
%timeit sum_parallel_chunk(a, num_chunks=100_000)

808 µs ± 147 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
327 µs ± 111 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
chunks
610 µs ± 48.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
423 µs ± 26.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
463 µs ± 90.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
520 µs ± 69.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
842 µs ± 28.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [90]:
def sum_divide(a):
  n = len(a)
  if n == 1:
    return a[0]
  else:
    split_index = math.ceil(n/2)
    left = a[:split_index]
    right = a[split_index:]
    return sum_divide(left) + sum_divide(right)

a = np.arange(100)
sum_divide(a), a.sum()

(4950, 4950)

In [98]:
def sum_reduce(a):
  n = len(a)
  a_cpy = a.copy()
  step_size = 1
  while step_size < n:
    for i in range(n):
      if i % (2 * step_size) == 0 and i + step_size < n:
        a_cpy[i] += a_cpy[i + step_size]
    step_size *= 2
  print(a_cpy)
  return a_cpy[0]

a = np.arange(7)
sum_reduce(a), a.sum()

[21  1  5  3 15  5  6]


(21, 21)

In [105]:
@njit(int32(int32[:]), parallel=True)
def sum_reduce_parallel(a):
  n = len(a)
  a_cpy = a.copy()
  step_size = 1
  while step_size < n:
    for i in prange(n): # 
      if i % (2 * step_size) == 0 and i + step_size < n:
        a_cpy[i] += a_cpy[i + step_size]
    step_size *= 2
  return a_cpy[0]

a = np.ones(100_000, dtype=np.int32)
%timeit sum_optimized(a)
%timeit sum_parallel(a)
%timeit sum_reduce_parallel(a)

58.5 µs ± 691 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
35.5 µs ± 1.42 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
2.49 ms ± 153 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [111]:
type(np.arange(10)[0])

numpy.int32

In [117]:
a = np.array([[0, 1], [1, 2]])

%timeit a[0, 0]
%timeit a[0][0]


152 ns ± 19.3 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
258 ns ± 10.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
